
# TP CNN interpretation: CAM and Guided Grad-CAM
### Diane LINGRAND 

diane.lingrand@univ-cotedazur.fr   
Octobre 2023

## Introduction

In [1]:
from IPython.display import Image
import tensorflow 
print(tensorflow.__version__)
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Activation
import matplotlib.pyplot as plt



2.15.0


**The GPU**



To enable GPU backend in Google colab for your notebook:

1.   Runtime (top left corner) -> Change runtime type
2.   Put GPU as "Hardware accelerator"
3.   Save.

Or run the next cell:

In [3]:
device_name = tensorflow.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


SystemError: GPU device not found

### Loading the dataset
In this part, we will use photographies of animals from the kaggle dataset [animals-10](https://www.kaggle.com/alessiocorrado99/animals10). Please connect to their site before loading the dataset from this [zip file](http://www.i3s.unice.fr/~lingrand/raw-img.zip). Decompress the zip file on your disk.

If you are using google colab, there is no need to download the dataset because I have a copy on my drive. You just need add to your drive this shared folder: https://drive.google.com/drive/folders/15cB1Ky-7OTUqfcQDZZyzc5HArt0GA6Sm?usp=sharing
You need to click on the link and click on "Add shortcut to Drive" and then select "My Drive".

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google.colab'

In [5]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D, Flatten
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, f1_score
import tensorflow.keras
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import glob
# when processing time is long, it's nice to see the progress bar
#!pip install tqdm
from tqdm import tqdm

ImportError: cannot import name 'plot_confusion_matrix' from 'sklearn.metrics' (C:\Users\praba\anaconda3\Lib\site-packages\sklearn\metrics\__init__.py)

### loading train data

Please read the code before running any of the cells!

In [ ]:
datasetRoot='/home/lingrand/Ens/MachineLearning/animals/raw-img/'
#datasetRoot='/whereYouPutTheImages/'
#datasetRoot='/content/drive/My Drive/raw-img/'
# I suggest to reduce the number of classes for a first trial. 
# If you finish this notebook before the end of the course, you can add more classes (and images per class).
classes = ['mucca', 'elefante', 'gatto'] #, 'cavallo', 'scoiattolo', 'ragno', 'pecora', 'farfalla', 'gallina', 'cane']
nbClasses = len(classes)

#training data

rootTrain = datasetRoot+'train/'
classLabel = 0
reducedSizePerClass = 200 #in order to reduce the number of images per class
totalImg = nbClasses * reducedSizePerClass
xTrain = np.empty(shape=(totalImg,224,224,3))
yTrain = []
first = True
i= 0
for cl in classes:
    listImages = glob.glob(rootTrain+cl+'/*')
    yTrain += [classLabel]*reducedSizePerClass #len(listImages) # note that here ...
    for pathImg in tqdm(listImages[:reducedSizePerClass]): # and here, we have reduced the data to be loaded (only 1000 per class)
        img = image.load_img(pathImg, target_size=(224,224))
        im = image.img_to_array(img)
        im = np.expand_dims(im, axis=0)
        im = preprocess_input(im)
        xTrain[i,:,:,:] = im
        i += 1
    classLabel += 1
print(len(yTrain))
print(xTrain.shape)
yTrain = tensorflow.keras.utils.to_categorical(yTrain, nbClasses)


In order to speed-up the time spent on this part of the lab, you may have noticed that we reduced the number of classes and the number of images per class. You can change these few lines of code if you want to work on the whole dataset.

### loading test data

In [ ]:
#you need to use the same classes for the test dataset than for the train dataset
rootTest = datasetRoot+'test/'
classLabel = 0

totalTestImg = 0
for cl in classes:
    totalTestImg += len(glob.glob(rootTest+cl+'/*'))

print("There are ",totalTestImg, " images in test dataset.")
xTest = np.empty(shape=(totalTestImg,224,224,3))
yTest = []
i = 0

for cl in classes:
    listImages = glob.glob(rootTest+cl+'/*')
    yTest += [classLabel]*len(listImages)
    for pathImg in listImages:
        img = image.load_img(pathImg, target_size=(224, 224))
        im = image.img_to_array(img)
        im = np.expand_dims(im, axis=0)
        im = preprocess_input(im)
        if(np.max(im)==0):
            print('n')
        xTest[i,:,:,:] = im.copy()
        if(np.max(xTest[i])==0):
            print('m')
        i+=1
    classLabel += 1
print(len(yTest))

yTest = tensorflow.keras.utils.to_categorical(yTest, nbClasses)

In [ ]:
i=135
img = image.load_img(listImages[i], target_size=(224, 224))
plt.imshow(img)
plt.show()

## CAM Class Activation Map

In [ ]:
VGGmodel = VGG16(weights='imagenet', include_top=False)
# we will add layers to this feature extraction part of VGG network
m = VGGmodel.output
# we start with a global average pooling
m = # your code
# finally, the softmax layer for predictions (we have nbClasses classes)
predictions = #layer (m)

# global network
model = Model(inputs=VGGmodel.input, outputs=predictions)


In [ ]:
model.summary()

In [ ]:
ourCallback = tensorflow.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=20, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

In [ ]:
# training part I: training only the classification part (the end)
for layer in VGGmodel.layers:
    layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(xTrain, yTrain, epochs=2000, batch_size=128, validation_split=0.2, callbacks=[ourCallback],verbose=1)

In [ ]:
# RECOVER THE WEIGHTS OF THE LAST LAYER
layer = model.get_layer(#the name of the last dense layer)
w = layer.get_weights()
print("number of weights: ", len(w))
w2 = w[0]
print(w2.shape)
# IS IT CORRECT ?

In [ ]:
# COMPUTE THE FEATURES MAPS FOR xTest
allFeatures = ###.predict(xTest)

In [ ]:
# PREDICT THE CLASSES FOR xTest
predClasses = ###.predict(xTest)

In [ ]:
xTest.shape

In [ ]:
index = # choose one index for one test image
maps = allFeatures[index]
print("dim of maps = ", maps.shape)

In [ ]:
#predicted class
cl = np.argmax(predClasses[index])
print("class = ", cl)

In [ ]:
we = ## extract the weights associated to the predicted class

In [ ]:
# COMPUTE THE CAM map
cam = 

In [ ]:
# rescale values of the cam map
leMin = np.min(cam)
leMax = np.max(cam)
camimg = (cam-leMin)/(leMax-leMin)
plt.imshow(cam, cmap = plt.cm.gray)


In [ ]:
# display the image in grey levels
plt.imshow(0.5*image[:,:,1],cmap=plt.cm.gray)
plt.show()

In [ ]:
print(image[:,:,1].shape)

In [ ]:
# and add the rescaled cam map with false colors and transparency
from skimage.transform import resize
plt.imshow(resize(camimg, (224,224)))
plt.imshow(image[:,:,1],cmap=plt.cm.gray, alpha=0.5)
plt.show()

## Grad-CAM

In [ ]:
import tensorflow as tf

In [ ]:
# example of gradient tape

with tf.GradientTape() as tape:
    output, predictions = model(np.array([img]))
    c = np.argmax(predictions)
    print("class = ", c)
    loss = predictions[:, cl]

In [ ]:
from skimage.transform import resize
plt.imshow(resize(gradcamimg, (224,224)))
plt.colorbar()
plt.imshow(image[:,:,1],cmap=plt.cm.gray, alpha=0.4)
plt.show

### Guided GradCAM